# Getting embeddings for reviews

In [1]:
import pandas as pd
import tiktoken
import openai
from openai import OpenAI

In [2]:
model = "text-embedding-3-small"
encoding = "cl100k_base"
max_tokens = 8000  # the maximum for text-embedding-3-small is 8191

In [3]:
client = OpenAI(api_key="")

def get_embedding(text, model = model):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], 
model=model).data[0].embedding

In [5]:
input_datapath = "datasets/extracted_db.csv"  # to save space, we provide a pre-filtered dataset
df = pd.read_csv(input_datapath, index_col=0)
df = df[["Review", "Name"]]
df = df.dropna()
df["combined"] = (
    "Name: " + df.Name.str.strip() + "; Review: " + df.Review.str.strip()
)

In [7]:
encoding = tiktoken.get_encoding(encoding)
df["n_tokens"] = df.combined.apply(lambda x: len(encoding.encode(x)))
df = df[df.n_tokens <= max_tokens]
df = df.groupby('Name').apply(lambda x: x.nlargest(5, 'n_tokens')).reset_index(drop=True)
len(df)

/var/folders/t5/qpsnpkfs3h527fshk0ntjbkr0000gn/T/ipykernel_19801/2388862166.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Name').apply(lambda x: x.nlargest(5, 'n_tokens')).reset_index(drop=True)


8968

In [ ]:
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, model=model))

In [ ]:
df.to_csv("datasets/reviews_with_embeddings.csv")